# Computing Word Overlap
## Step 1
This week’s lab provides practice in word counting over multiple files. More specifically,
we look at computing the level of word overlap between documents as an indicator of
document similarity, which might arise in circumstances such as document duplication,
text reuse (e.g. plagiarism), even simply having the same topic.

## Step 2
Download the file `NEWS.zip` from the [module homepage](https://staffwww.dcs.shef.ac.uk/people/M.Hepple/campus_only/COM6115/) (see: **Lab Classes > Data/code
files**). Unzip this to get a folder `NEWS` of short news articles (with names `news01.txt`, `news02.txt`, etc). Also download files `compare_STARTER_CODE.py` and `stop_list.txt`, and place them *alongside* the unzipped news folder. A *stop-list* is a list of 'unimportant' words - words that are often ignored during word counting (discussed later in the course).

In [1]:
import os
if 'NEWS.zip' not in os.listdir():
    import requests
    url = 'http://jupi.ink:8889/notebooks/COM6115%20Text%20Processing/Practice/NEWS.zip'
    r = requests.get(url) 
    with open("NEWS.zip",'wb') as f:
        f.write(r.content)
if 'NEWS' not in os.listdir():
    assert 'NEWS.zip' in os.listdir()
    print('Extract files...')
    import zipfile
    zip = zipfile.ZipFile('./NEWS.zip', 'r')
    for name in zip.namelist():
        zip.extract(name, '.')
    print('Finished.')

## Step 3
Rename the *starter* code file as `compare.py`, and extend it to complete the lab exercises.

The script might be called from the command line as follows:
```shell
> python compare.py -s stop_list.txt NEWS/news01.txt NEWS/news02.txt
```
Here, flag **-s** identifies `stop_list.txt` as an (optional) stoplist. The subsequent command line arguments (`news01.txt` and `news02.txt`) identify the files to be compared. These file names are stored as a list in the variable filenames.

The starter code loads the stoplist, storing its words as a set in the variable stops. This is an appropriate choice of data structure, as it allows fast, hash-based look-up, via a simple test such as “**w in stops**”. If the stoplist is not specified, **stops** will store an empty set, in which case, the same test can be used, but will always return `False`.

Later, we will want to call the code to take all the files in the `NEWS` folder as input, but the way to do this depends on the operating system (OS) being used.
UNIX/LINUX/MAC: simply use ‘wildcards’ in the filename, as in the following example:
```shell
> python compare.py -s stop_list.txt NEWS/news??.txt
```
WINDOWS: the above convenience is not available in the Windows terminal, so the code
instead implements it via the -I option, which must be supplied with a pattern to match
the file names, as in the following example:
```shell
> python compare.py -s stop_list.txt -I NEWS/news??.txt
```
Study the starter code provided, to make sure you understand it.


In [2]:
!python compare.py -s stop_list.txt NEWS/news??.txt

INPUT-FILES: NEWS/news01.txt NEWS/news02.txt NEWS/news03.txt NEWS/news04.txt NEWS/news05.txt NEWS/news06.txt NEWS/news07.txt NEWS/news08.txt NEWS/news09.txt NEWS/news10.txt NEWS/news11.txt NEWS/news12.txt NEWS/news13.txt NEWS/news14.txt NEWS/news15.txt NEWS/news16.txt NEWS/news17.txt NEWS/news18.txt NEWS/news19.txt NEWS/news20.txt NEWS/news21.txt NEWS/news22.txt NEWS/news23.txt NEWS/news24.txt NEWS/news25.txt NEWS/news26.txt NEWS/news27.txt NEWS/news28.txt NEWS/news29.txt NEWS/news30.txt NEWS/news31.txt NEWS/news32.txt


## Step 4
Starting with the case where there are only `two input files`, extend the code to compute
and print a word `overlap score` (explained below in 7.) between the two texts, e.g. as in:

In [3]:
!python compare.py -s stop_list.txt NEWS/news01.txt NEWS/news02.txt

INPUT-FILES: NEWS/news01.txt NEWS/news02.txt


Some suggestions about components to implement as part of this script follow.

## Step 5
TOKENISATION: To simplify, you might treat the words as being all the maximal alphabetic sequences found in the file (which can readily be extracted from each line with
a simple regex, and the regex `findall` method). To improve **overlap**, you should convert
the text to lowercase, so that different case variants are conflated in counting.

In [4]:
def Readfile(path='NEWS/news{:02}.txt'.format(1)):
    with open(path) as handle:
        text = handle.read()
    return text

## Step 6
COUNTING: You might define a function (e.g. `count_words`) which counts the occurrences of (non-stoplist) words in a file, and returns these counts as a **dictionary**, i.e. with
**words as keys**, and **counts as values**. (Alternatively, you might define a simple class, whose instances store this information, and have the word counting function as a method.)

In [5]:
class words:
    def __init__(self, text:str):
        self.text = text
        self.stop = stop
        self.dict = dict()
        self.CountWords()
    
    def _add2dict(self,word:str):
        self.dict[word] = self.dict.get(word, 0) + 1
    
    def _text2list(self,text:str) -> list:
        import re
        return re.findall(r'[a-z]+',text.lower())
    
    def CountWords(self):
        stop_set = set(self._text2list(self.stop))
        words_list = self._text2list(self.text)
        for word in words_list:
            if word not in stop_set:
                self._add2dict(word)

## Step 7
COMPARISON: You might define a function which, given a dictionary of the counts for
two files, computes a measure of their word overlap. We will use versions of the *jaccard
coefficient*. A simple version ignores the counts of words in the files, and instead treats
each as just a set of word types. For word sets A and B, this measure is computed as:
$$
\frac{\left | A\cap B \right |}{\left | A\cup B \right |}
$$

In [6]:
def Jaccard(a:dict,b:dict)->float:
    cap = set(a) & set(b)
    cup = set(a) | set(b)
    return len(cap)/len(cup) if len(cup) != 0 else 0

stop = Readfile('NEWS/stop_list.txt')
A = words(Readfile('NEWS/news01.txt'))
B = words(Readfile('NEWS/news02.txt'))
score = Jaccard(A.dict,B.dict)
print('Jaccard score between {} and {} is {:.3}'\
      .format('NEWS/news01.txt','NEWS/news02.txt',score))

Jaccard score between NEWS/news01.txt and NEWS/news02.txt is 0.0199


## Step 8
For a count-sensitive version of a jaccard metric, we might use the following. For compared files A and B, let $w_A$ and $w_B$ denote the counts of word $w$ in the two files respectively.
Our measure is then computed as follows (where $\sum_{w\in A\cup B}$ here ranges over the full set of terms appearing in *either* file):
$$
\frac{\sum_{w\in A\cup B}min(w_A,w_B)}{\sum_{w\in A\cup B}max(w_A,w_B)}
$$
You might add a command-line option to determine whether this metric or the previous (*count-insensitive*) metric is applied, e.g. a boolean option **-b** (for binary $0$/$1$ weighting)

In [7]:
def JaccardSens(a:dict,b:dict)->float:
    up,down = 0,0
    for word in set(a) | set(b):
        w_a = a.get(word,0)
        w_b = b.get(word,0)
        up += min(w_a,w_b)
        down += max(w_a,w_b)
    return up/down
score_sens = JaccardSens(A.dict,B.dict)
print('Jaccard sensitive score between {} and {} is {:.3}'\
      .format('NEWS/news01.txt','NEWS/news02.txt',score_sens))

Jaccard sensitive score between NEWS/news01.txt and NEWS/news02.txt is 0.0167


## Step 9
Having produced a script that can compare two files, you might modify it to perform pairwise comparison across more than two files, e.g. as in the following, where the pattern picks out three input files for comparison:
```shell
> python compare.py -s stop_list.txt -I ’NEWS/news0[123].txt’
NEWS/news01.txt <> NEWS/news02.txt = 0.017
NEWS/news01.txt <> NEWS/news03.txt = 0.000
NEWS/news02.txt <> NEWS/news03.txt = 0.011
```
Given the number of comparisons made here, you might modify your code to store up
the scores, and then print out only the top N (e.g. 10) scoring comparisons.

In [8]:
def Combination(L):
    total = []
    while len(L) > 1:
        a = L.pop(0) # get number
        s = [[a,b] for b in L] # get combination
        [total.append(i) for i in s] # save combination
    return total

def Multi_file(filename):
    import re
    files_index = list(re.findall('\[([0-9]+)\]','NEWS/news0[123].txt')[0])
    for a,b in Combination(files_index):
        file_a_name = 'NEWS/news{:02}.txt'.format(int(a))
        file_b_name = 'NEWS/news{:02}.txt'.format(int(b))
        A = words(Readfile(file_a_name))
        B = words(Readfile(file_b_name))
        score_sens = JaccardSens(A.dict,B.dict)
        print('{}  <>  {} = {:.2}'\
              .format(file_a_name,file_b_name,score_sens))

Multi_file('NEWS/news0[123].txt')

NEWS/news01.txt  <>  NEWS/news02.txt = 0.017
NEWS/news01.txt  <>  NEWS/news03.txt = 0.0
NEWS/news02.txt  <>  NEWS/news03.txt = 0.011


## Step 10
Apply your code to the full set of news articles to see if you can find the following cases
of related files:
* *duplication*: two of the news files are identical, in terms of their textual content
(although they are not literally identical, as might be tested using unix diff)
* *plagiarism*: one of the news files has been produced by cutting and pasting together
text from two of the other files
* *related topics*: three of the articles address the same news story, as given by three
different newspapers. Do these separately authored presentations of the same story
exhibit higher overlap than articles on unrelated topics? (Note, two of these related
articles appear within the first 5 news stories.)